In [2]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper


api_wrapper_wiki = WikipediaAPIWrapper(top_k_results = 1,doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)

api_arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_arxiv_wrapper)

In [4]:
tools = [wiki,arxiv]

In [6]:
## Custom Tools
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

loader = WebBaseLoader(web_path="https://python.langchain.com/docs/integrations/text_embedding/ollama/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(separators="\n\n",chunk_size = 1000,chunk_overlap = 200)
chunks = documents.split_documents(docs)
embeddings = OllamaEmbeddings(model="llama3.1",base_url="http://localhost:11434/")
vector_bd = FAISS.from_documents(chunks,embeddings)
retriever = vector_bd.as_retriever()

In [8]:
from langchain.tools.retriever import create_retriever_tool

retriever_chain = create_retriever_tool(retriever,"Custom_Langsmith-search","Search any information about Langchain")
retriever_chain

Tool(name='Custom_Langsmith-search', description='Search any information about Langchain', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002878448D440>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000028787FC1B90>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x00000287849A2520>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000028787FC1B90>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'))

In [9]:
tools = [wiki,arxiv,retriever_chain]

In [10]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Raghu\\OneDrive\\Documents\\Anaconda\\envs\\LSTM\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='Custom_Langsmith-search', description='Search any information about Langchain', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002878448D440>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS ob

In [11]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
os.environ["GROQ_API"] = os.getenv("GROQ_API")
groq_api_key = os.getenv("GROQ_API")

In [12]:
llm = ChatGroq(model="llama3-groq-70b-8192-tool-use-preview",api_key=groq_api_key)

In [14]:
response = llm.invoke("As an aspiring gen AI developer, Do I really need to pursue DSA and leetcode?")
response.content

"Yes, as an aspiring general AI developer, pursuing Data Structures and Algorithms (DSA) and practicing on platforms like LeetCode can be beneficial in several ways:\n\n1. Foundational Knowledge: DSA provides the foundational knowledge required for understanding and developing efficient algorithms, which are crucial for building AI systems. It helps you understand how to process and manipulate data efficiently, leading to better performance and scalability in your AI models.\n\n2. Problem-Solving Skills: Practicing on platforms like LeetCode improves your problem-solving skills, which are essential for AI development. You'll learn to approach complex problems systematically, break them down into smaller parts, and find efficient solutions.\n\n3. Understanding Computational Complexity: DSA helps you understand computational complexity, which is critical in AI. You'll learn to analyze the time and space complexity of algorithms, allowing you to make informed decisions when choosing or de

In [23]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)

In [26]:
response = agent_executor({"input": "Tell me more about OllamaEmbeddings"})
print(response)

C:\Users\Raghu\AppData\Local\Temp\ipykernel_5968\1513065577.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent_executor({"input": "Tell me more about OllamaEmbeddings"})


TypeError: Agent.plan() got multiple values for argument 'intermediate_steps'